# Movies Reviews Sentiment Analysis

In [1]:
%load_ext autoreload
%autoreload 2

import util

/home/andres/anaconda2/envs/as-kaggle/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


### Carga de datos

In [2]:
train, dev, test = util.load_datasets_unlabeled_test()
X_train, y_train = train
X_dev, y_dev = dev
from collections import Counter
Counter(y_train), Counter(y_dev), len(test)

(Counter({1: 482, 0: 481}), Counter({1: 53, 0: 54}), 500)

Podemos ver que para la *train* y *dev* tenemos a la clase equilibrada (al rededor de 50% de entradas positivas y negativas), y el conjunto de *dev* tiene el 10% de la cantidad que tiene el conjunto de *train*.

In [3]:
import pandas as pd
baseline = pd.read_csv('review_polarity_competition/results_baseline.csv')
Counter(baseline['Category'])

Counter({0: 273, 1: 227})

Para el conjunto de *test* no tenemos la distribución de positivos y negativos, pero en el *baseline* que tiene al rededor de 86% de accuracy, las categorías están igualmente distribuidas, por lo que se puede asumir que la distribución del conjunto de *test* es igual a la del conjunto de *dev*.

### Baseline

Ya tenemos los resultados de un modelo como punto de referencia, en donde se logra un *accuracy* del 86%, por lo tanto los prócimos modelos tienen que buscar superar esa performance.

### Vectorización

Para la vectorización de los datos vamos a probar con `CountVectorizer` y con `TfidfVectorizer`.

Para medir la performance de cada vectorizador, vamos a utilizar por el momento el modelo de `LogisticRegression`, con los parámetros por defecto.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import numpy as np

np.random.seed(42)

vects = [
    CountVectorizer(),
    TfidfVectorizer(),
]

param_grid = {
    'vect__binary': [True, False],
    'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'vect__min_df': [3, 5, 7],
    'vect__max_df': [0.95, 0.9, 0.7],
    'clf__random_state': [0],
}

for vect in vects:
    pipeline = Pipeline([
        ('vect', vect),
        ('clf', LogisticRegression()),
    ])
    model = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
    model.fit(X_train, y_train)
    
    print("# Exploración de parámetros para \"%s\"" % vect, end="\n\n")
    
    print("Mejor conjunto de parámetros:")
    print(model.best_params_, end="\n\n")

    print("Puntajes de la grilla:", end="\n\n")
    means = model.cv_results_['mean_test_score']
    stds = model.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, model.cv_results_['params']):
        print("Exactitud: %0.3f (+/-%0.03f) para los parámetros" % (mean, std ** 2))
    print()
    
    print("Reporte de clasificación para el mejor clasificador (sobre conjunto de evaluación):", end="\n\n")
    print(util.print_eval(model, X_dev, y_dev), end="\n\n")

# Exploración de parámetros para "CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)"

Mejor conjunto de parámetros:
{'clf__random_state': 0, 'vect__binary': True, 'vect__max_df': 0.95, 'vect__min_df': 3, 'vect__ngram_range': (1, 3)}

Puntajes de la grilla:

Exactitud: 0.834 (+/-0.000) para los parámetros
Exactitud: 0.837 (+/-0.000) para los parámetros
Exactitud: 0.840 (+/-0.000) para los parámetros
Exactitud: 0.834 (+/-0.000) para los parámetros
Exactitud: 0.837 (+/-0.000) para los parámetros
Exactitud: 0.832 (+/-0.000) para los parámetros
Exactitud: 0.829 (+/-0.000) para los parámetros
Exactitud: 0.828 (+/-0.000) para los parámetros
Exactitud: 0.828 (+/-0.000) para 

In [5]:
model.best_params_

{'clf__random_state': 0,
 'vect__binary': True,
 'vect__max_df': 0.9,
 'vect__min_df': 5,
 'vect__ngram_range': (1, 1)}

No se observan grandes diferencias en los resultados. Si bien en el conjunto de *dev* se observan mejores resultados para `CountVectorizer`, en el *cross-validation* se pueden ver resultados generales mejores con la vectorización de `TfidfVectorizer`, por lo que optamos por esa forma de vectorizar. 

### Exploración de Modelo

A partir de la mejor forma de vectorización obtenida, vamos a probar diferentes modelos.

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier

clfs = [
    KNeighborsClassifier(),
    MultinomialNB(),
    DecisionTreeClassifier(random_state=0),
    LogisticRegression(random_state=0),
    LinearSVC(random_state=0),
    SVC(random_state=0),
    RandomForestClassifier(random_state=0),
]

vect = TfidfVectorizer(binary=True,
                      max_df=0.9,
                      min_df=5,
                      ngram_range=(1,1))

for clf in clfs:
    print(str(clf.__class__))
    pipeline = Pipeline([
        ('vect', vect),
        ('clf', clf),
    ])
    pipeline.fit(X_train, y_train)
    util.print_short_eval(pipeline, X_train, y_train)
    util.print_short_eval(pipeline, X_dev, y_dev)

<class 'sklearn.neighbors.classification.KNeighborsClassifier'>


/home/andres/anaconda2/envs/as-kaggle/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


accuracy	0.88	macro f1	0.88
accuracy	0.80	macro f1	0.80
<class 'sklearn.naive_bayes.MultinomialNB'>
accuracy	0.94	macro f1	0.94
accuracy	0.85	macro f1	0.85
<class 'sklearn.tree.tree.DecisionTreeClassifier'>
accuracy	1.00	macro f1	1.00
accuracy	0.68	macro f1	0.68
<class 'sklearn.linear_model.logistic.LogisticRegression'>
accuracy	0.97	macro f1	0.97
accuracy	0.88	macro f1	0.88
<class 'sklearn.svm.classes.LinearSVC'>
accuracy	1.00	macro f1	1.00
accuracy	0.85	macro f1	0.85
<class 'sklearn.svm.classes.SVC'>


/home/andres/anaconda2/envs/as-kaggle/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


accuracy	0.50	macro f1	0.33
accuracy	0.50	macro f1	0.33
<class 'sklearn.ensemble.forest.RandomForestClassifier'>
accuracy	0.99	macro f1	0.99
accuracy	0.73	macro f1	0.72


A partir de estos resultados vamos ajustando los modelos que mejores resultados dan.

#### LogisticRegression

In [7]:
import numpy as np
from sklearn.model_selection import GridSearchCV

param_grid = {
    'clf__tol' : np.logspace(-6,-4,3),
    'clf__C' : np.logspace(-1,1,3),
    'clf__max_iter' : np.logspace(2,3,2)
}

pipeline = Pipeline([
    ('vect', vect),
    ('clf', LogisticRegression(random_state=0)),
])
model = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
model.fit(X_train, y_train)

print("# Exploración de parámetros", end="\n\n")

print("Mejor conjunto de parámetros:")
print(model.best_params_, end="\n\n")

print("Puntajes de la grilla:", end="\n\n")
means = model.cv_results_['mean_test_score']
stds = model.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, model.cv_results_['params']):
    print("Exactitud: %0.3f (+/-%0.03f) para los parámetros %r" % (mean, std ** 2, params))
print()

print("Reporte de clasificación para el mejor clasificador (sobre conjunto de evaluación):", end="\n\n")
print(util.print_eval(model, X_dev, y_dev), end="\n\n")

# Exploración de parámetros

Mejor conjunto de parámetros:
{'clf__C': 1.0, 'clf__max_iter': 100.0, 'clf__tol': 1e-06}

Puntajes de la grilla:

Exactitud: 0.830 (+/-0.001) para los parámetros {'clf__C': 0.1, 'clf__max_iter': 100.0, 'clf__tol': 1e-06}
Exactitud: 0.830 (+/-0.001) para los parámetros {'clf__C': 0.1, 'clf__max_iter': 100.0, 'clf__tol': 1e-05}
Exactitud: 0.830 (+/-0.001) para los parámetros {'clf__C': 0.1, 'clf__max_iter': 100.0, 'clf__tol': 0.0001}
Exactitud: 0.830 (+/-0.001) para los parámetros {'clf__C': 0.1, 'clf__max_iter': 1000.0, 'clf__tol': 1e-06}
Exactitud: 0.830 (+/-0.001) para los parámetros {'clf__C': 0.1, 'clf__max_iter': 1000.0, 'clf__tol': 1e-05}
Exactitud: 0.830 (+/-0.001) para los parámetros {'clf__C': 0.1, 'clf__max_iter': 1000.0, 'clf__tol': 0.0001}
Exactitud: 0.857 (+/-0.001) para los parámetros {'clf__C': 1.0, 'clf__max_iter': 100.0, 'clf__tol': 1e-06}
Exactitud: 0.857 (+/-0.001) para los parámetros {'clf__C': 1.0, 'clf__max_iter': 100.0, 'clf__tol': 1e-

In [8]:
import pickle
filename = 'logreg.p'
f = open(filename, 'wb')
pickle.dump(model, f)

### LinearSVC

In [9]:
param_grid = {
    'clf__tol' : np.logspace(-6,-4,3),
    'clf__C' : np.logspace(-2,1,4),
}

pipeline = Pipeline([
    ('vect', vect),
    ('clf', LinearSVC(random_state=0)),
])
model = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
model.fit(X_train, y_train)

print("# Exploración de parámetros", end="\n\n")

print("Mejor conjunto de parámetros:")
print(model.best_params_, end="\n\n")

print("Puntajes de la grilla:", end="\n\n")
means = model.cv_results_['mean_test_score']
stds = model.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, model.cv_results_['params']):
    print("Exactitud: %0.3f (+/-%0.03f) para los parámetros %r" % (mean, std ** 2, params))
print()

print("Reporte de clasificación para el mejor clasificador (sobre conjunto de evaluación):", end="\n\n")
print(util.print_eval(model, X_dev, y_dev), end="\n\n")

# Exploración de parámetros

Mejor conjunto de parámetros:
{'clf__C': 0.1, 'clf__tol': 1e-06}

Puntajes de la grilla:

Exactitud: 0.826 (+/-0.001) para los parámetros {'clf__C': 0.01, 'clf__tol': 1e-06}
Exactitud: 0.826 (+/-0.001) para los parámetros {'clf__C': 0.01, 'clf__tol': 1e-05}
Exactitud: 0.826 (+/-0.001) para los parámetros {'clf__C': 0.01, 'clf__tol': 0.0001}
Exactitud: 0.856 (+/-0.001) para los parámetros {'clf__C': 0.1, 'clf__tol': 1e-06}
Exactitud: 0.856 (+/-0.001) para los parámetros {'clf__C': 0.1, 'clf__tol': 1e-05}
Exactitud: 0.856 (+/-0.001) para los parámetros {'clf__C': 0.1, 'clf__tol': 0.0001}
Exactitud: 0.854 (+/-0.000) para los parámetros {'clf__C': 1.0, 'clf__tol': 1e-06}
Exactitud: 0.854 (+/-0.000) para los parámetros {'clf__C': 1.0, 'clf__tol': 1e-05}
Exactitud: 0.854 (+/-0.000) para los parámetros {'clf__C': 1.0, 'clf__tol': 0.0001}
Exactitud: 0.838 (+/-0.001) para los parámetros {'clf__C': 10.0, 'clf__tol': 1e-06}
Exactitud: 0.838 (+/-0.001) para los parámet

In [10]:
import pickle
filename = 'linearsvc.p'
f = open(filename, 'wb')
pickle.dump(model, f)

### RandomForest

In [11]:
param_grid = {
    'clf__n_estimators' : [100, 500, 1000],
    'clf__criterion' : ['gini', 'entropy'],
    'clf__max_features' : ['sqrt', 'log2', 10, 50]
}

pipeline = Pipeline([
    ('vect', vect),
    ('clf', RandomForestClassifier(random_state=0)),
])
model = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
model.fit(X_train, y_train)

print("# Exploración de parámetros", end="\n\n")

print("Mejor conjunto de parámetros:")
print(model.best_params_, end="\n\n")

print("Puntajes de la grilla:", end="\n\n")
means = model.cv_results_['mean_test_score']
stds = model.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, model.cv_results_['params']):
    print("Exactitud: %0.3f (+/-%0.03f) para los parámetros %r" % (mean, std ** 2, params))
print()

print("Reporte de clasificación para el mejor clasificador (sobre conjunto de evaluación):", end="\n\n")
print(util.print_eval(model, X_dev, y_dev), end="\n\n")

# Exploración de parámetros

Mejor conjunto de parámetros:
{'clf__criterion': 'entropy', 'clf__max_features': 50, 'clf__n_estimators': 500}

Puntajes de la grilla:

Exactitud: 0.757 (+/-0.001) para los parámetros {'clf__criterion': 'gini', 'clf__max_features': 'sqrt', 'clf__n_estimators': 100}
Exactitud: 0.777 (+/-0.001) para los parámetros {'clf__criterion': 'gini', 'clf__max_features': 'sqrt', 'clf__n_estimators': 500}
Exactitud: 0.775 (+/-0.002) para los parámetros {'clf__criterion': 'gini', 'clf__max_features': 'sqrt', 'clf__n_estimators': 1000}
Exactitud: 0.752 (+/-0.001) para los parámetros {'clf__criterion': 'gini', 'clf__max_features': 'log2', 'clf__n_estimators': 100}
Exactitud: 0.767 (+/-0.002) para los parámetros {'clf__criterion': 'gini', 'clf__max_features': 'log2', 'clf__n_estimators': 500}
Exactitud: 0.763 (+/-0.002) para los parámetros {'clf__criterion': 'gini', 'clf__max_features': 'log2', 'clf__n_estimators': 1000}
Exactitud: 0.759 (+/-0.001) para los parámetros {'clf

In [12]:
import pickle
filename = 'randomforest.p'
f = open(filename, 'wb')
pickle.dump(model, f)

### MultinomialNB

In [13]:
param_grid = {
    'clf__alpha' : [1, 0.7, 0.9, 0.95, 0.85],
}

pipeline = Pipeline([
    ('vect', vect),
    ('clf', MultinomialNB()),
])
model = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
model.fit(X_train, y_train)

print("# Exploración de parámetros", end="\n\n")

print("Mejor conjunto de parámetros:")
print(model.best_params_, end="\n\n")

print("Puntajes de la grilla:", end="\n\n")
means = model.cv_results_['mean_test_score']
stds = model.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, model.cv_results_['params']):
    print("Exactitud: %0.3f (+/-%0.03f) para los parámetros %r" % (mean, std ** 2, params))
print()

print("Reporte de clasificación para el mejor clasificador (sobre conjunto de evaluación):", end="\n\n")
print(util.print_eval(model, X_dev, y_dev), end="\n\n")

# Exploración de parámetros

Mejor conjunto de parámetros:
{'clf__alpha': 0.9}

Puntajes de la grilla:

Exactitud: 0.850 (+/-0.000) para los parámetros {'clf__alpha': 1}
Exactitud: 0.848 (+/-0.001) para los parámetros {'clf__alpha': 0.7}
Exactitud: 0.852 (+/-0.000) para los parámetros {'clf__alpha': 0.9}
Exactitud: 0.852 (+/-0.000) para los parámetros {'clf__alpha': 0.95}
Exactitud: 0.852 (+/-0.000) para los parámetros {'clf__alpha': 0.85}

Reporte de clasificación para el mejor clasificador (sobre conjunto de evaluación):

accuracy	0.85

             precision    recall  f1-score   support

        neg       0.83      0.89      0.86        54
        pos       0.88      0.81      0.84        53

avg / total       0.85      0.85      0.85       107

[[48  6]
 [10 43]]
None



In [14]:
import pickle
filename = 'multi.p'
f = open(filename, 'wb')
pickle.dump(model, f)

### KNeighborsClassifier

In [15]:
param_grid = {
    'clf__n_neighbors' : [2, 5, 10, 15, 20, 30, 40, 50, 60, 70],
    'clf__weights' : ['distance'],
    'clf__algorithm' : ['brute'],
}

pipeline = Pipeline([
    ('vect', vect),
    ('clf', KNeighborsClassifier()),
])
model = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
model.fit(X_train, y_train)

print("# Exploración de parámetros", end="\n\n")

print("Mejor conjunto de parámetros:")
print(model.best_params_, end="\n\n")

print("Puntajes de la grilla:", end="\n\n")
means = model.cv_results_['mean_test_score']
stds = model.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, model.cv_results_['params']):
    print("Exactitud: %0.3f (+/-%0.03f) para los parámetros %r" % (mean, std ** 2, params))
print()

print("Reporte de clasificación para el mejor clasificador (sobre conjunto de evaluación):", end="\n\n")
print(util.print_eval(model, X_dev, y_dev), end="\n\n")

# Exploración de parámetros

Mejor conjunto de parámetros:
{'clf__algorithm': 'brute', 'clf__n_neighbors': 50, 'clf__weights': 'distance'}

Puntajes de la grilla:

Exactitud: 0.737 (+/-0.002) para los parámetros {'clf__algorithm': 'brute', 'clf__n_neighbors': 2, 'clf__weights': 'distance'}
Exactitud: 0.769 (+/-0.001) para los parámetros {'clf__algorithm': 'brute', 'clf__n_neighbors': 5, 'clf__weights': 'distance'}
Exactitud: 0.794 (+/-0.001) para los parámetros {'clf__algorithm': 'brute', 'clf__n_neighbors': 10, 'clf__weights': 'distance'}
Exactitud: 0.813 (+/-0.000) para los parámetros {'clf__algorithm': 'brute', 'clf__n_neighbors': 15, 'clf__weights': 'distance'}
Exactitud: 0.815 (+/-0.001) para los parámetros {'clf__algorithm': 'brute', 'clf__n_neighbors': 20, 'clf__weights': 'distance'}
Exactitud: 0.832 (+/-0.001) para los parámetros {'clf__algorithm': 'brute', 'clf__n_neighbors': 30, 'clf__weights': 'distance'}
Exactitud: 0.840 (+/-0.001) para los parámetros {'clf__algorithm': 'br

In [16]:
import pickle
filename = 'knn.p'
f = open(filename, 'wb')
pickle.dump(model, f)

### Ensemble

A partir de los modelos generados se pretende analizar los resultados de considerarlos a todos los modelos con algún método de *ensemble* como `VotingClassifier`

In [17]:
file = open("logreg.p",'rb')
logreg = pickle.load(file)
file = open("linearsvc.p",'rb')
linearsvc = pickle.load(file)
file = open("randomforest.p",'rb')
randomforest = pickle.load(file)
file = open("multi.p",'rb')
multi = pickle.load(file)
file = open("knn.p",'rb')
knn = pickle.load(file)

In [31]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'weights' : [[5, 4, 1, 3, 2]],
}

ensemble = VotingClassifier(estimators=[('logreg', logreg), ('linearsvc', linearsvc), ('randomforest', randomforest), ('multi', multi), ('knn', knn)],
                         voting='hard')
model = GridSearchCV(ensemble, param_grid, cv=5, scoring='accuracy')
model.fit(X_train, y_train)

print("# Exploración de parámetros", end="\n\n")

print("Mejor conjunto de parámetros:")
print(model.best_params_, end="\n\n")

means = model.cv_results_['mean_test_score']
stds = model.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, model.cv_results_['params']):
    print("Exactitud: %0.3f (+/-%0.03f) para los parámetros %r" % (mean, std ** 2, params))
print()

print("Reporte de clasificación para el mejor clasificador (sobre conjunto de evaluación):", end="\n\n")
print(util.print_eval(model, X_dev, y_dev), end="\n\n")

/home/andres/anaconda2/envs/as-kaggle/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/andres/anaconda2/envs/as-kaggle/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/andres/anaconda2/envs/as-kaggle/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/andres/anaconda2/envs/as-kaggle/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151

# Exploración de parámetros

Mejor conjunto de parámetros:
{'weights': [5, 4, 1, 3, 2]}

Exactitud: 0.849 (+/-0.001) para los parámetros {'weights': [5, 4, 1, 3, 2]}

Reporte de clasificación para el mejor clasificador (sobre conjunto de evaluación):

accuracy	0.88

             precision    recall  f1-score   support

        neg       0.89      0.87      0.88        54
        pos       0.87      0.89      0.88        53

avg / total       0.88      0.88      0.88       107

[[47  7]
 [ 6 47]]
None



/home/andres/anaconda2/envs/as-kaggle/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [32]:
util.save_results('results4.csv', model.predict(test))

/home/andres/anaconda2/envs/as-kaggle/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
